# Segment Trees

A Segment Tree is a data structure that stores information about array intervals as a tree. This allows answering range queries over an array efficiently, while still being flexible enough to allow quick modification of the array.

This includes finding the sum of consecutive array elements a[l..r], or finding the minimum element in a such a range in O(log(N)) time.

One important property of Segment Trees is that they require only a linear amount of memory. The standard Segment Tree requires 4N vertices for working on an array of size N.

## Simplest form of a Segment Tree

We want to answer sum queries efficiently. The formal definition of our task is: Given an array A[0..N-1], the Segment Tree must be able to find the sum of elements between the indices l and r (i.e. computing the sum  
sum(A for i=l,r), and also handle changing values of the elements in the array (i.e. perform assignments of the form A[i] = x ). The Segment Tree should be able to process both queries in O(log(N)) time.

This is an improvement over the simpler approaches. A naive array implementation - just using a simple array - can update elements in O(1), but requires O(N) to compute each sum query. And precomputed prefix sums can compute sum queries in O(1), but updating an array element requires O(N) changes to the prefix sums.

### Structure of the Segment Tree
We can take a divide-and-conquer approach when it comes to array segments. We compute and store the sum of the elements of the whole array, i.e. the sum of the segment A[0..N-1]. We then split the array into two halves A[0..N/2-1] and A[N/2..N-1] and compute the sum of each halve and store them. Each of these two halves in turn are split in half, and so on until all segments reach size 1.
![sum-segment-tree.png](https://cp-algorithms.com/data_structures/sum-segment-tree.png)

The number of vertices in the worst case can be estimnated by the sum, 1+2+4+...+2**(logN + 1) < 2**(logN + 1) < 4n.

It is worth noting that whenever  
N is not a power of two, not all levels of the Segment Tree will be completely filled. We can see that behavior in the image.

The height of the Segment Tree is  
O(logN) , because when going down from the root to the leaves the size of the segments decreases approximately by half.

### Construction
Before constructing the segment tree, we need to decide:

1. the value that gets stored at each node of the segment tree. For example, in a sum segment tree, a node would store the sum of the elements in its range [l, r].
2. the merge operation that merges two siblings in a segment tree. For example, in a sum segment tree, the two nodes corresponding to the ranges A[l1...r1]] and A[l2...r2] would be merged into a node corresponding to the range A[l1...r2] by adding the values of the two nodes.

Note that a vertex is a "leaf vertex", if its corresponding segment covers only one value in the original array. It is present at the lowermost level of a segment tree. Its value would be equal to the (corresponding) element A[i].

The time complexity of this construction is O(N), assuming that the merge operation is constant time (the merge operation gets called N times, which is equal to the number of internal nodes in the segment tree).

### Sum Queries
As an input we receive two integers l and we have to compute the sum of the segment  A[l...r] in O(logN) time.

To do this, we will traverse the Segment Tree and use the precomputed sums of the segments. Let's assume that we are currently at the vertex that covers the segment A[tl...tr]. There are three possible cases.
1. The easiest case is when the segment A[l...r] is equal to the corresponding segment of the current vertex (i.e.
A[l...r] == A[tl...tr]), then we are finished and can return the precomputed sum that is stored in the vertex.
2. Alternatively the segment of the query can fall completely into the domain of either the left or the right child. Recall that the left child covers the segment  A[tl...tm]  and the right vertex covers the segment A[tm+1...tr] with tm = (tl+tr) / 2. In this case we can simply go to the child vertex, which corresponding segment covers the query segment, and execute the algorithm described here with that vertex.
3. And then there is the last case, the query segment intersects with both children. In this case we have no other option as to make two recursive calls, one for each child. First we go to the left child, compute a partial answer for this vertex (i.e. the sum of values of the intersection between the segment of the query and the segment of the left child), then go to the right child, compute the partial answer using that vertex, and then combine the answers by adding them.

The calculation of the query is a traversal of the tree, which spreads through all necessary branches of the tree, and uses the precomputed sum values of the segments in the tree.

Why is the complexity of this algorithm O(logN)? To show this complexity we look at each level of the tree. It turns out, that for each level we only visit not more than the height of the tree. And since the height of the tree is O(logN), we receive the desired running time.

### Update Queries
Now we want to modify a specific element in the array, let's say we want to do the assignment A[i] = x. And we have to rebuild the Segment Tree, such that it corresponds to the new, modified array.

Therefore an element A[i] only contributes to one segment from each level. Thus only O(logN) vertices need to be updated.

It is easy to see, that the update request can be implemented using a recursive function. The function gets passed the current tree vertex, and it recursively calls itself with one of the two child vertices (the one that contains A[i] in its segment), and after that recomputes its sum value, similar how it is done in the build method (that is as the sum of its two children).

### Implementation

In [15]:
class SegmentTree:
    def __init__(self, arr):
        self.arr_len = len(arr)
        self.segment_array = [0] * (4 * self.arr_len)
        self.seg_arr_len = 4 * self.arr_len
        self.arr = arr

    def build(self, vertex_idx, tl, tr):
        if tl == tr:
            self.segment_array[vertex_idx] = self.arr[tl]
            return

        tm = (tr - tl) // 2 + tl
        self.build(vertex_idx * 2 + 1, tl, tm)
        self.build(vertex_idx * 2 + 2, tm+1, tr)

        self.segment_array[vertex_idx] = self.segment_array[vertex_idx * 2 + 1] + self.segment_array[vertex_idx * 2 + 2]

    # Query Call
    def sum(self, vertex_idx, tl, tr, query_l, query_r):
        if query_l > query_r:
            return 0
        elif query_l == tl and query_r == tr:
            return self.segment_array[vertex_idx]

        tm = (tr - tl) // 2 + tl

        left = self.sum(vertex_idx * 2 + 1, tl, tm, query_l, min(query_r, tm)) 
        right = self.sum(vertex_idx * 2 + 2, tm+1, tr, max(tm+1, query_l), query_r)
        return left + right

    def update(self, vertex_idx, tl, tr, position, new_val):
        if tl == tr:
            self.segment_array[vertex_idx] = new_val 
            return 

        tm = (tr - tl) // 2 + tl

        if position <= tm:
            self.update(vertex_idx * 2 + 1, tl, tm, position, new_val)
        else:
            self.update(vertex_idx * 2 + 2, tm+1, tr, position, new_val)

        self.segment_array[vertex_idx] = self.segment_array[vertex_idx * 2 + 1] + self.segment_array[vertex_idx * 2 + 2]


# Example
arr = [5, 3, 6, 2, 1, 7]
st = SegmentTree(arr)
st.build(0, 0, len(arr)-1)

print(st.sum(0, 0, len(arr)-1, 3, 5))
st.update(0, 0, len(arr)-1, 3, 6)
print(st.sum(0, 0, len(arr)-1, 3, 5))

10
14
